In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [ ]:
df.sort_values(by=['Price'], ascending=False)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
3952,4079,Land Rover Range Rover 3.0 Diesel LWB Vogue,Hyderabad,2017,25000,Diesel,Automatic,First,13.33 kmpl,2993 CC,255 bhp,5.0,2.3 Cr,160.00
5620,5781,Lamborghini Gallardo Coupe,Delhi,2011,6500,Petrol,Automatic,Third,6.4 kmpl,5204 CC,560 bhp,2.0,NaN,120.00
5752,5919,Jaguar F Type 5.0 V8 S,Hyderabad,2015,8000,Petrol,Automatic,First,12.5 kmpl,5000 CC,488.1 bhp,2.0,NaN,100.00
1457,1505,Land Rover Range Rover Sport SE,Kochi,2019,26013,Diesel,Automatic,First,12.65 kmpl,2993 CC,255 bhp,5.0,1.39 Cr,97.07
1917,1974,BMW 7 Series 740Li,Coimbatore,2018,28060,Petrol,Automatic,First,12.05 kmpl,2979 CC,320 bhp,5.0,NaN,93.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,3228,Maruti 800 Std,Pune,2003,52000,Petrol,Manual,First,16.1 kmpl,796 CC,37 bhp,4.0,NaN,0.50
1577,1628,Maruti 800 Std BSIII,Jaipur,2004,12000,Petrol,Manual,Second,16.1 kmpl,796 CC,37 bhp,4.0,NaN,0.45
2758,2847,Hyundai Getz GLS,Pune,2005,86000,Petrol,Manual,Second,15.3 kmpl,1341 CC,83 bhp,5.0,NaN,0.45
3039,3138,Maruti Zen LXI,Jaipur,1998,95150,Petrol,Manual,Third,17.3 kmpl,993 CC,60 bhp,5.0,NaN,0.45


In [ ]:
print(df.isna().sum())
print('')
print(df.isnull().sum())

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [ ]:
"""
Given the small number of missing values and the fact that their contents can't
be infered, I opt to drop all of the rows with missing values from Mileage,
Engine, and Power.
"""
dfV1 = df.dropna(subset=['Mileage', 'Engine', 'Power']).copy()
dfV1.isna().sum()

"""
Since the information about the number of seats should be consistent for cars of
the same year and model, we can replace the missing values with ones from other
cars of the same model.
"""
dfV1["Seats"] = (df.groupby(["Name"])["Seats"].transform(lambda x : x.mean()))

#print(dfV1[dfV1["Seats"].isna()])
#print(dfV1[dfV1["Name"] == "Ford Figo Diesel"])

"""
The remaining missing value in Seats is for a car that does not have any other
examples, so I opt to drop it as well.
"""

dfV1 = dfV1.dropna(subset=['Seats'])
dfV1.isna().sum()


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


In [ ]:
# As for the remaining missing entries in New Price, there are too many to remove
# every row missing an entry so I will fill the available entries with the median for
# vehicles of the same Name.

dfV1[['New Price Number', 'New Price Unit']] = dfV1['New_Price'].str.extract(r'([-]?[.0-9]+)\s*([a-zA-Z/]+)')

print(dfV1['New_Price'].head())
print(dfV1['New Price Unit'].head())
dfV1['New Price Number'] = pd.to_numeric(dfV1['New Price Number'])

dfV1.loc[dfV1['New Price Unit'] == 'Cr', 'New Price Number'] = dfV1.loc[dfV1['New Price Unit'] == 'Cr', 'New Price Number'] * 100

dfV1 = dfV1.drop(['New_Price'], axis = 1)
dfV1 = dfV1.drop(['New Price Unit'], axis = 1)

0          NaN
1    8.61 Lakh
2          NaN
3          NaN
4          NaN
Name: New_Price, dtype: object
0     NaN
1    Lakh
2     NaN
3     NaN
4     NaN
Name: New Price Unit, dtype: object


In [ ]:
# Since there are still a significant amount of missing entries, it would be
# impractical to completely remove every row missing a value. The best thing to
# do in this case would seem to be using the mean value of the New_Price column
# to impute the missing values.

dfV1['New Price in Lakhs'] = dfV1['New Price Number'].fillna(dfV1['New Price Number'].mean())
dfV1 = dfV1.drop(['New Price Number'], axis = 1)
dfV1['New Price in Lakhs'].astype(float)
print(dfV1.isna().sum())

Unnamed: 0            0
Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine                0
Power                 0
Seats                 0
Price                 0
New Price in Lakhs    0
dtype: int64


In [ ]:
"""
Since the Mileage contains a mixture of units which depend on the type of fuel,
so the units and number must be extracted from the string and be used to convert
the row to a single unit. I choose to convert all entries to km/kg.
"""
dfV1[['Mileage Number', 'Mileage Unit']] = dfV1['Mileage'].str.extract(r'([-]?[.0-9]+)\s*([a-zA-Z/]+)')

dfV1['Mileage Number'] = pd.to_numeric(dfV1['Mileage Number'])

# Convert Diesel 'kmpl' to 'km/kg' (assuming 0.85 as specific gravity for Diesel)
dfV1.loc[(dfV1['Mileage Unit'] == 'kmpl') & (dfV1['Fuel_Type'] == 'Diesel'), 'Mileage Number'] = \
    dfV1.loc[(dfV1['Mileage Unit'] == 'kmpl') & (dfV1['Fuel_Type'] == 'Diesel'), 'Mileage Number'] / 0.85

# Convert Petrol 'kmpl' to 'km/kg' (assuming 0.75 as specific gravity for Petrol)
dfV1.loc[(dfV1['Mileage Unit'] == 'kmpl') & (dfV1['Fuel_Type'] == 'Petrol'), 'Mileage Number'] = \
    dfV1.loc[(dfV1['Mileage Unit'] == 'kmpl') & (dfV1['Fuel_Type'] == 'Petrol'), 'Mileage Number'] / 0.75

print(dfV1['Mileage'].head())
print(dfV1['Mileage Number'].head())
dfV1 = dfV1.drop(['Mileage'], axis = 1)
dfV1 = dfV1.drop(['Mileage Unit'], axis = 1)
dfV1 = dfV1.rename(columns={'Mileage Number': 'Mileage in km/kg'})

0    19.67 kmpl
1      13 km/kg
2    20.77 kmpl
3     15.2 kmpl
4    23.08 kmpl
Name: Mileage, dtype: object
0    23.141176
1    13.000000
2    24.435294
3    17.882353
4    27.152941
Name: Mileage Number, dtype: float64


In [ ]:
dfV1['Engine in CC'] = dfV1['Engine'].str.replace(r'CC', '', regex=True)
dfV1['Engine in CC'] = pd.to_numeric(dfV1['Engine in CC'])
dfV1 = dfV1.drop(['Engine'], axis = 1)

In [ ]:
dfV1['Power in bhp'] = dfV1['Power'].str.replace(r'bhp', '', regex=True)
dfV1['Power in bhp'] = pd.to_numeric(dfV1['Power in bhp'])
dfV1 = dfV1.drop(['Power'], axis = 1)

In [ ]:
dfOHE = pd.get_dummies(dfV1, columns=["Fuel_Type","Transmission"])
dfOHE["Diesel Fuel"] = dfOHE["Fuel_Type_Diesel"].astype(int)
dfOHE["Petrol Fuel"] = dfOHE["Fuel_Type_Petrol"].astype(int)
dfOHE["Manual Transmission"] = dfOHE["Transmission_Manual"].astype(int)
dfOHE["Automatic Transmission"] = dfOHE["Transmission_Automatic"].astype(int)
dfOHE = dfOHE.drop(['Fuel_Type_Diesel', 'Fuel_Type_Petrol','Transmission_Manual', 'Transmission_Automatic'], axis=1)

In [ ]:
dfOHE['Kilometers per year'] = dfOHE['Kilometers_Driven'].astype(int) /(2025 - dfOHE['Year'])

In [ ]:
print(dfOHE.head())

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Seats  Price  New Price in Lakhs  \
0              41000      First    5.0  12.50            23.28183   
1              46000      First    5.0   4.50             8.61000   
2              87000      First    7.0   6.00            23.28183   
3              40670     Second    5.0  17.74            23.28183   
4              86999      First    5.0   3.50            23.28183   

   Mileage in km/kg  Engine in CC  Power in bhp  Diesel Fuel  Petrol Fuel  \
0         23.141176          1582        126.20            1            0   
1         13.000

In [ ]:
dfOHE.describe()

,Unnamed: 0,Year,Kilometers_Driven,Seats,Price,New Price in Lakhs,Mileage in km/kg,Engine in CC,Power in bhp,Diesel Fuel,Petrol Fuel,Manual Transmission,Automatic Transmission,Kilometers per year
count,5808.000000,5808.000000,5.808000e+03,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000,5808.000000
mean,3013.497762,2013.474346,5.835413e+04,5.286501,9.672056,23.281830,22.680116,1631.815427,113.825253,0.542700,0.457300,0.706956,0.293044,5043.856901
std,1737.870169,3.172395,9.264840e+04,0.806279,11.291525,9.831284,5.271463,601.773588,53.899159,0.498216,0.498216,0.455198,0.455198,10995.033482
min,1.000000,1998.000000,1.710000e+02,2.000000,0.440000,3.910000,0.000000,624.000000,34.200000,0.000000,0.000000,0.000000,0.000000,28.500000
25%,1508.750000,2012.000000,3.329000e+04,5.000000,3.590000,23.281830,19.040000,1198.000000,78.000000,0.000000,0.000000,0.000000,0.000000,3181.730303
50%,3016.500000,2014.000000,5.241550e+04,5.000000,5.750000,23.281830,23.141176,1497.000000,98.600000,1.000000,0.000000,1.000000,0.000000,4587.117647
75%,4518.250000,2016.000000,7.246600e+04,5.000000,10.250000,23.281830,26.266667,1991.000000,139.025000,1.000000,1.000000,1.000000,1.000000,6052.744444
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000,230.000000,34.666667,5998.000000,560.000000,1.000000,1.000000,1.000000,1.000000,812500.000000
